# NGO 87/102

In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Funde

In [2]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Ort.ort_kurz,
           t_Komplex.bef_nr, 
           t_Obj.Individuum, 
           t_Obj.GefEinheit, 
           t_Obj.Typ, 
           t_Obj.Anzahl,
           t_Obj.Gewicht,
           t_Obj.Gr_Clist, 
           t_Obj.Art,
           t_Obj.Fabric,
           t_Obj.Form_Gef
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) Like '%NGO%')
           AND ((t_Komplex.bef_nr) Like '%87/102%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,ort_kurz,bef_nr,Individuum,GefEinheit,Typ,Anzahl,Gewicht,Gr_Clist,Art,Fabric,Form_Gef
0,458,NGO,87/102,28-29,1,NGO,1,1914,500,K,1d,G10a
1,1248,NGO,87/102,2,5,NGO,1,84,120,K,1d,G12a
2,1249,NGO,87/102,"19,21,29,31",4,NGO,1,623,200,K,1d,G2a
3,1250,NGO,87/102,31,,,1,40,120,K,None,
4,1515,NGO,87/102,27,3,NGO,1,379,200,K,1d,G1a2
5,1516,NGO,87/102,30,2,NGO,1,525,500,K,1d,G12a


In [6]:
df_pivot1 = df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot1 = pd.DataFrame(df_pivot1)
# df_pivot1 = df_pivot1.reset_index()
df_pivot1.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot1['\%'] = (df_pivot1['Gewicht']/df_pivot1['Gewicht'].sum()*100).round(1)
df_pivot1['Gewicht'] = (df_pivot1['Gewicht'] / 1000).round(2)
df_pivot1.rename(columns={'Gewicht': 'Gewicht (kg)'}, inplace=True)

df_pivot2 = df.pivot_table(values = 'Anzahl', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot2 = pd.DataFrame(df_pivot2)
df_pivot2.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot2['\%'] = (df_pivot2['Anzahl']/df_pivot2['Anzahl'].sum()*100).round(1)

# Tabellen vereinen:
df_concat = pd.concat([df_pivot2, df_pivot1], axis = 1)
df_concat = df_concat.reset_index()
df_concat.rename(columns={'Art': 'Fundkategorie'}, inplace=True)
# df_concat.to_latex('../output/tabs/9-08_PIK871-1_Funde.tex', index = False, escape = False)
df_concat

,Fundkategorie,Anzahl,\%,Gewicht (kg),\%
0,Keramik,6,100,3.56,100


In [3]:
df['Gewicht'].sum()

3565

In [4]:
df.pivot_table(values = 'objID', index = 'Form_Gef', aggfunc = len)

Form_Gef
        1
G10a    1
G12a    2
G1a2    1
G2a     1
Name: objID, dtype: int64

In [5]:
df.pivot_table(values = 'objID', index = 'Fabric', aggfunc = len)

Fabric
1d    5
Name: objID, dtype: int64